In [1]:
from pathlib import Path
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [2]:
from open_pref_eval.evaluation import evaluate_model
from open_pref_eval.plot.radar import radar_plot
from open_pref_eval.datasets.genies import dist2datasets, GENIES
from open_pref_eval.datasets.ethics import get_ethics_datasets
from open_pref_eval.datasets import load_dataset_n
from open_pref_eval.datasets import ds2name
from open_pref_eval.plot.radar import radar_plot

In [3]:
import pandas as pd
from reprpo.models.load import load_model, print_trainable_parameters

In [4]:
f = Path('/workspace/repr-preference-optimization/outputs/princeton-nlp_Llama-3-Base-8B-SFT_HRAKL_us_history_textbook/2024-09-18_10-34-31-adapter/HRAKL-us_history_textbook')
sorted(f.glob('**/*'))

# TODO turn into a df we can filter

[]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from peft import AutoPeftModelForCausalLM, get_peft_model, PeftConfig, PeftModelForCausalLM

In [76]:
out_dir = Path('../outputs/').resolve()
import json
f = out_dir.glob('princeton-nlp_Llama-3-Base-8B-SFT*/**/adapter_config.json')

def path2dict(p):
    assert p.is_dir()
    d1 = p.name # adapter - ds
    d2 = p.parent.parent.name # date
    d3 = p.parent.parent.parent.name # base method ds
    print(d3)
    adapter = d1
    time = d2
    # base_method, adapter2, dsname2 = str(d3).split('_', 2)
    # adapter2, dsname = dsname2.split('_', 1)

    # base_model = PeftConfig.from_pretrained(p).base_model_name_or_path
    c = json.load((p.parent.parent/'config.json').open())

    return dict(adapter= adapter,
                # adapter2 = adapter2,
                dsname = c['dataset'],
                # dsname2 = dsname2,
                base_model = c['base_model'],
                time = time,
                path = p)



df_paths = pd.DataFrame([path2dict(p.parent) for p in f])
df_paths

princeton-nlp_Llama-3-Base-8B-SFT_projbp_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_textbook
princeton-nlp_Llama-3-Base-8B-SFT_projgrad_us_history_tex

,adapter,dsname,base_model,time,path
0,projbp,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_10-38-39,/workspace/repr-preference-optimization/output...
1,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_03-16-31,/workspace/repr-preference-optimization/output...
2,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_03-47-31,/workspace/repr-preference-optimization/output...
3,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_04-18-06,/workspace/repr-preference-optimization/output...
4,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_04-42-54,/workspace/repr-preference-optimization/output...
5,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_04-51-11,/workspace/repr-preference-optimization/output...
6,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_05-25-58,/workspace/repr-preference-optimization/output...
7,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_05-53-25,/workspace/repr-preference-optimization/output...
8,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_06-10-34,/workspace/repr-preference-optimization/output...
9,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29_06-27-45,/workspace/repr-preference-optimization/output...


In [85]:
df_paths.time = pd.to_datetime(df_paths.time, format='%Y-%m-%d_%H-%M-%S')
df_paths = df_paths.sort_values('time')
df_paths

,adapter,dsname,base_model,time,path
1,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 03:16:31,/workspace/repr-preference-optimization/output...
2,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 03:47:31,/workspace/repr-preference-optimization/output...
3,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 04:18:06,/workspace/repr-preference-optimization/output...
4,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 04:42:54,/workspace/repr-preference-optimization/output...
5,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 04:51:11,/workspace/repr-preference-optimization/output...
6,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 05:25:58,/workspace/repr-preference-optimization/output...
7,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 05:53:25,/workspace/repr-preference-optimization/output...
8,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 06:10:34,/workspace/repr-preference-optimization/output...
9,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 06:27:45,/workspace/repr-preference-optimization/output...
10,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 06:45:42,/workspace/repr-preference-optimization/output...


In [94]:
base_model = df_paths.base_model.unique()[0]
base_model

df_paths = df_paths.query('base_model == @base_model')
df_paths

,adapter,dsname,base_model,time,path
1,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 03:16:31,/workspace/repr-preference-optimization/output...
2,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 03:47:31,/workspace/repr-preference-optimization/output...
3,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 04:18:06,/workspace/repr-preference-optimization/output...
4,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 04:42:54,/workspace/repr-preference-optimization/output...
5,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 04:51:11,/workspace/repr-preference-optimization/output...
6,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 05:25:58,/workspace/repr-preference-optimization/output...
7,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 05:53:25,/workspace/repr-preference-optimization/output...
8,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 06:10:34,/workspace/repr-preference-optimization/output...
9,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 06:27:45,/workspace/repr-preference-optimization/output...
10,projgrad,us_history_textbook,princeton-nlp/Llama-3-Base-8B-SFT,2024-09-29 06:45:42,/workspace/repr-preference-optimization/output...


In [90]:
fs = df_paths.groupby(['adapter', 'dsname']).last().path.values

In [77]:
from reprpo.interventions.config import ExperimentConfig
args = ExperimentConfig(verbose=True)

In [99]:
f = fs[0]
peft_config = PeftConfig.from_pretrained(f)
tokenizer = AutoTokenizer.from_pretrained(base_model)
_base_model = AutoModelForCausalLM.from_pretrained(f)
model = PeftModelForCausalLM.from_pretrained(
    _base_model,
    f, config=peft_config, adapter_name=f.stem)
model.peft_config

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [126]:
f.stem

'projgrad'

In [100]:
# load all other adapters
for f in fs[1:]:
    model.load_adapter(f, f.stem)
model.peft_config

{'default': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='princeton-nlp/Llama-3-Base-8B-SFT', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=64, target_modules={'v_proj', 'q_proj'}, lora_alpha=16, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=True, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False)),
 'dpo': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='princeton-nlp/Llama-3-Base-8B-SFT', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=64, target_modules={'v_proj', 'q_proj'}, lora_alpha=16, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=True, modules_to_save=None, init_lora_weights=True, layers

In [102]:
model = model.cuda()

In [103]:

# ## Eval
# eval on ethics, GENIES, and our train dataset
N = None
N = 256
dataset = 'us_history_textbook'
datasets = [
    load_dataset_n('wassname/genies_preferences', name=args.dataset, split='train', N=N),
    load_dataset_n('wassname/genies_preferences', name=args.dataset, split='test', N=N),
]
datasets += dist2datasets(GENIES, N=N, source=[args.dataset]) # our hard OOS test
datasets += get_ethics_datasets(N=N)
datasets += [load_dataset_n('wassname/genies_preferences', name=name, split='test', N=N) for name in ['math_make_questions', 'truthful_qa', 'wrong_arc', 'ranking_logic',
 'math', 'sycophancy_mimicry'
]]

Generating train split:   0%|          | 0/1800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/750 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1513 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/652 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/606 [00:00<?, ? examples/s]

...

Generating test split:   0%|          | 0/750 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/750 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

In [104]:
datasets

[Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 64
 }),
 Dataset({
     features: ['prompt', 'chosen', 'reje

In [105]:
res, df_res2 = evaluate_model(model=model, 
                                tokenizer=tokenizer, 
                                datasets=datasets,
                                    batch_size=args.batch_size*2,
                                    bf16=True,
                                    torch_empty_cache_steps=100, verbowse=args.verbose)

  0%|          | 0/13 [00:00<?, ?dataset/s]

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [106]:
from collections import OrderedDict
ds_alias = OrderedDict(
    list(zip(["train", "test", "oos", "rnd"], [ds2name(d) for d in datasets]))
)

In [152]:
from reprpo.training import parse_eval
from reprpo.gen import get_model_generations, display_gen

def rename_columns(df):
    df.columns = [d.replace("genie_dpo-", "") for d in df.columns]
    df.columns = [d.replace("genies_preferences-", "") for d in df.columns]
    df.columns = [d.replace("ethics_expression_preferences-", "") for d in df.columns]
    df.columns = [d.split(f"[:{N}]")[0] for d in df.columns]
    df.columns = [d.split(f"-test")[0] for d in df.columns]
    return df

In [153]:
adapter_name = df_res2[["adapter"]].query('adapter!="base"').values[0, 0]

df_res = (
    df_res2.groupby(["dataset", "adapter"], dropna=False)["correct"]
    .mean()
    .unstack()
    .T
)
df_res = rename_columns(df_res)

print('accuracy')
df_res

accuracy


,commonsense,deontology,justice,utilitarianism,math,math_make_questions,ranking_logic,sycophancy_mimicry,truthful_qa,us_history_textbook,us_history_textbook-train,wrong_arc,us_history_fiction
adapter,,,,,,,,,,,,,
base,0.671875,0.625000,0.359375,0.421875,0.859375,0.984375,0.593750,0.328125,0.546875,1.0,1.0,0.265625,0.750000
default,0.656250,0.625000,0.312500,0.437500,0.890625,0.984375,0.656250,0.234375,0.781250,1.0,1.0,0.140625,0.812500
dpo,0.656250,0.625000,0.312500,0.437500,0.890625,0.984375,0.656250,0.234375,0.781250,1.0,1.0,0.140625,0.812500
projbp,0.656250,0.640625,0.343750,0.421875,0.906250,0.968750,0.671875,0.218750,0.718750,1.0,1.0,0.156250,0.890625
projgrad,0.687500,0.609375,0.296875,0.421875,0.906250,0.968750,0.640625,0.187500,0.765625,1.0,1.0,0.140625,0.843750


In [154]:
df_res_logp = (
    df_res2.groupby(["dataset", "adapter"], dropna=False)["_chosen_ppl"]
    .mean()
    .unstack()
    .T
)
df_res_logp = rename_columns(df_res_logp)
print('perplexity of chosen response')
df_res_logp

perplexity of chosen response


,commonsense,deontology,justice,utilitarianism,math,math_make_questions,ranking_logic,sycophancy_mimicry,truthful_qa,us_history_textbook,us_history_textbook-train,wrong_arc,us_history_fiction
adapter,,,,,,,,,,,,,
base,4.011998e+03,202.782104,259.957062,2.900816e+02,3.340670,5.468517,18.129017,6.490664,4.416421e+01,3.326440,3.194985,1.409406e+03,3.093051
default,2.922118e+06,6725.777832,43791.367188,1.772942e+05,287.549133,1312.053833,29622.550781,857.691040,6.026577e+09,2382.266846,1735.492188,1.336507e+14,1628.402710
dpo,2.922118e+06,6725.777832,43791.367188,1.772942e+05,287.549133,1312.053833,29622.550781,857.691040,6.026577e+09,2382.266846,1735.492188,1.336507e+14,1628.402710
projbp,4.639360e+07,9275.056641,33628.765625,7.364031e+05,202.406525,657.913757,17236.373047,1085.653931,7.351039e+07,1241.406372,1221.444092,5.175616e+14,437.433929
projgrad,2.192658e+06,6798.495117,99186.109375,1.963471e+07,48.358810,229.375946,548.673828,388.404022,1.534295e+05,101.440453,76.747536,9.441994e+11,92.343445


In [155]:
df_res2["logratios"] = df_res2["_chosen_logps"] - df_res2["_rejected_logps"]
df_logratios = (
    df_res2.groupby(["dataset", "adapter", "ds_i"])["logratios"].mean().unstack(0)
)
ref_logratios = df_logratios.loc["base"]
preference_logp_gain_vs_ref = (df_logratios - ref_logratios)#.mean('ds_i')
preference_logp_gain_vs_ref = preference_logp_gain_vs_ref.groupby('adapter').mean()
preference_logp_gain_vs_ref = rename_columns(preference_logp_gain_vs_ref)
print('how much chosen response is prefered')
preference_logp_gain_vs_ref

dpo of chosen response


,commonsense,deontology,justice,utilitarianism,math,math_make_questions,ranking_logic,sycophancy_mimicry,truthful_qa,us_history_textbook,us_history_textbook-train,wrong_arc,us_history_fiction
adapter,,,,,,,,,,,,,
base,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
default,3.123202,4.864337,-7.912639,-0.761393,21.573036,79.290901,6.123348,-29.201908,25.938736,322.582062,365.516296,-28.241354,162.247421
dpo,3.123202,4.864337,-7.912639,-0.761393,21.573036,79.290901,6.123348,-29.201908,25.938736,322.582062,365.516296,-28.241354,162.247421
projbp,3.856024,5.367977,-6.655376,-0.903116,20.462229,76.824722,6.827622,-23.482504,24.038534,311.224731,329.704010,-22.393742,183.724777
projgrad,4.484490,5.238854,-8.555972,-1.706129,17.713858,63.965103,3.830432,-36.790810,23.083588,255.557251,263.538116,-25.465775,129.078232
